In [22]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)


In [ ]:
df = pd.read_json("data/joined_full_movies.json")

In [24]:
df.isna().sum()
df.columns
df.isna().sum()

jpbox_fr_title                  0
jpbox_released_year             0
allocine_fr_title               0
allocine_released_year          0
jpbox_original_title            0
jpbox_actors                    0
jpbox_producers                 0
jpbox_authors                   0
jpbox_compositors               0
jpbox_directors                 0
jpbox_country                   0
jpbox_category                  0
jpbox_released_date             0
jpbox_classification            0
jpbox_duration                  0
jpbox_total_entrances           0
jpbox_weekly_entrances          0
jpbox_incomes_total          1263
jpbox_incomes_france         2621
jpbox_budget                 1263
jpbox_url_movie                 0
jpbox_synopsis                461
allocine_original_title      1570
allocine_url                 1570
allocine_released_date       1570
allocine_duration            1575
allocine_categories          1570
allocine_writer              1570
allocine_director            1570
allocine_casti

In [25]:
df = df[['jpbox_fr_title', 'allocine_fr_title', 'jpbox_released_year',  
        'jpbox_actors', 'allocine_casting', 
        'jpbox_directors', 'allocine_director', 
        'allocine_writer', 'jpbox_authors', 
        'allocine_distribution', 
        'jpbox_country', "jpbox_budget",
        'jpbox_category',
        'jpbox_released_date', 
        'jpbox_classification', 'allocine_classification', 
        'jpbox_duration', 
        'jpbox_weekly_entrances']]

In [26]:
# traitement directors
df["jpbox_directors"] = df["jpbox_directors"].apply(lambda x : x[0] if len(x) > 0 else None)
df["jpbox_directors"] = df["jpbox_directors"].fillna(df["allocine_director"])
df = df.drop("allocine_director", axis=1)


In [ ]:
# traitement actor
df["jpbox_actors"] = df["jpbox_actors"].apply(lambda x : x if x and len(x) > 0 else np.nan)
df["jpbox_actors"] = df["jpbox_actors"].fillna(df["allocine_casting"])

df = df.drop("allocine_casting", axis=1)


In [28]:
df["jpbox_actors"].value_counts()

jpbox_actors
[]                                                              50
[Ariane Ascaride, Jean-Pierre Darroussin, Gérard Meylan]         5
[Shia LaBeouf, Megan Fox, Josh Duhamel]                          3
[Jean-Paul Rouve, Isabelle Nanty, Claire Nadeau]                 3
[Mark Hamill, Harrison Ford, Carrie Fisher]                      3
                                                                ..
[Mathias Legoût Hammond, Céleste Brunnquell, Élodie Bouchez]     1
[Reda Kateb, Marc Barbé, Farida Rahouadj]                        1
[Debbie Doebereiner, Dustin Ashley, Misty Wilkins]               1
[Baran Kosari, Amir Jadidi, Sahar Dowlatshahi]                   1
[Stephanie Cumming, Christoph Bach, Florentin Groll]             1
Name: count, Length: 6176, dtype: int64

In [7]:
# classification
df["allocine_classification"] = df["allocine_classification"].fillna(df["jpbox_classification"]).fillna("Tout public")
df = df.drop("jpbox_classification", axis = 1)
df["allocine_classification"] = df["allocine_classification"].replace({
    "Tous publics": "Tout public",
    "" : "Tout public",
    "Tout public avec avertissement": "Tout public",
    "à partir de 3 ans": "Tout public", 
    "Moins de 18 ans" : "Interdit - 18 ans", 
    "Interdit - 12 ans avec avertissement" : "Interdit - 12 ans",
    "Moins de 12 ans" : "Interdit - 12 ans",
    "Interdit - 16 ans avec avertissement" : "Interdit - 16 ans",
    "Moins de 16 ans" : "Interdit - 16 ans",
    "à partir de 10 ans" : "Interdit - 10 ans",
    "à partir de 6 ans" : "Interdit - 6 ans",
    "à partir de 8 ans" : "Interdit - 8 ans"
})


In [8]:
# authors
df["jpbox_authors"] = df["jpbox_authors"].apply(lambda x : x[0] if isinstance(x, list) and len(x) > 0 else None)
df["allocine_writer"] = df["allocine_writer"].apply(lambda x : x[0] if isinstance(x, list) and len(x) > 0 else None)
df["allocine_writer"] = df["allocine_writer"].fillna(df["jpbox_authors"])
df = df.drop("jpbox_authors", axis = 1)

In [9]:
# budget

df["jpbox_budget"] = df["jpbox_budget"].str.replace(r"[^\d]", "", regex=True)


In [10]:
def duration_minutes(duration):
    if pd.isna(duration):
        return np.nan
    try:
        duration = duration.replace("h", "").replace("min", "").strip()
        parts = duration.split()
        heures = int(parts[0]) if len(parts) > 0 else 0
        minutes = int(parts[1]) if len(parts) > 1 else 0
        return heures * 60 + minutes
    except:
        return np.nan

df["duration_minutes"] = df["jpbox_duration"].apply(duration_minutes)

df["duration_minutes"] = df["duration_minutes"].fillna(round(df["duration_minutes"].mean(),0))


In [11]:
df[df["jpbox_duration"] == "Aventure"]

,jpbox_fr_title,allocine_fr_title,jpbox_released_year,jpbox_actors,jpbox_directors,allocine_writer,allocine_distribution,jpbox_country,jpbox_budget,jpbox_category,jpbox_released_date,allocine_classification,jpbox_duration,jpbox_weekly_entrances,duration_minutes


In [12]:
df.groupby("allocine_classification")["allocine_classification"].count()

allocine_classification
Interdit - 10 ans     182
Interdit - 12 ans     766
Interdit - 16 ans     127
Interdit - 18 ans       6
Interdit - 6 ans      232
Interdit - 8 ans      201
Tout public          6336
Name: allocine_classification, dtype: int64

In [13]:
# jpbox category

df["jpbox_category"]

0                 Comédie
1                 Comédie
2         Science Fiction
3               Comicbook
4       Aventure - Action
              ...        
7845    Aventure - Action
7846    Aventure - Action
7847        Film familial
7848            Comicbook
7849            Animation
Name: jpbox_category, Length: 7850, dtype: object

In [14]:
df.head(10)

,jpbox_fr_title,allocine_fr_title,jpbox_released_year,jpbox_actors,jpbox_directors,allocine_writer,allocine_distribution,jpbox_country,jpbox_budget,jpbox_category,jpbox_released_date,allocine_classification,jpbox_duration,jpbox_weekly_entrances,duration_minutes
0,La Ch’tite famille,La Ch’tite famille,2018,"[Dany Boon, François Berléand, Valérie Bonneto...",Dany Boon,Dany Boon,Pathé Films,France,27800000,Comédie,28/02/2018,Tout public,1h 47min,2429906,107.0
1,Rien à déclarer,Rien à déclarer,2011,"[Dany Boon, Benoît Poelvoorde, François Damien...",Dany Boon,Dany Boon,Pathé Films,France,24434009,Comédie,02/02/2011,Interdit - 10 ans,1h 48min,2587056,108.0
2,Avatar,Avatar,2009,"[Sam Worthington, Sigourney Weaver]",James Cameron,James Cameron,The Walt Disney Company France,Etats-Unis,315000000,Science Fiction,16/12/2009,Tout public,2h 40min,2648596,160.0
3,Avengers: Infinity War,Avengers: Infinity War,2018,"[Chris Evans, Chris Hemsworth, Josh Brolin, Ro...",- Russo (brothers),Christopher Markus,The Walt Disney Company France,Etats-Unis,295000000,Comicbook,25/04/2018,Tout public,2h 36min,2565953,156.0
4,"Pirates des Caraïbes, le secret du coffre maudit",Pirates des Caraïbes : le Secret du Coffre Maudit,2006,"[Johnny Depp, Orlando Bloom, Geoffrey Rush, Ke...",Gore Verbinski,Ted Elliott,Buena Vista International,Etats-Unis,225000000,Aventure - Action,02/08/2006,Interdit - 10 ans,2h 30min,2708112,150.0
5,Harry Potter et les reliques de la mort - part...,Harry Potter et les reliques de la mort - part...,2010,"[Daniel Radcliffe, Emma Watson, Helena Bonham ...",David Yates,Joanne Kathleen Rowling,None,Etats-Unis,125000000,Fantasy,24/11/2010,Tout public,2h 30min,2537450,150.0
6,Le Roi Lion (2019),Le Roi Lion,2019,"[Jamel Debbouze, Jean Reno, Seth Rogen]",Jon Favreau,Jeff Nathanson,The Walt Disney Company France,Etats-Unis,260000000,Aventure - Action,17/07/2019,Interdit - 6 ans,1h 58min,3252896,118.0
7,Harry Potter et le prisonnier d'Azkaban,Harry Potter et le Prisonnier d'Azkaban,2004,"[Daniel Radcliffe, Emma Watson, Gary Oldman, A...",Alfonso Cuaron,Steve Kloves,Warner Bros. France,Etats-Unis,130000000,Fantasy,02/06/2004,Interdit - 10 ans,2h 21min,2503699,141.0
8,Star Wars: Les derniers Jedi,Star Wars - Les Derniers Jedi,2017,"[Adam Driver, Benicio Del Toro, Joseph Gordon-...",Rian Johnson,Rian Johnson,The Walt Disney Company France,Etats-Unis,200000000,Fantasy,13/12/2017,Interdit - 8 ans,2h 31min,2510462,151.0
9,Harry Potter et l'Ordre du Phénix,Harry Potter et l'Ordre du Phénix,2007,"[Robert Pattinson, Daniel Radcliffe, Emma Wats...",David Yates,Michael Goldenberg,Warner Bros. France,Etats-Unis,150000000,Fantasy,11/07/2007,Interdit - 10 ans,2h 18min,2486291,138.0


In [15]:
df.isna().sum()

jpbox_fr_title                0
allocine_fr_title             0
jpbox_released_year           0
jpbox_actors                633
jpbox_directors            1144
allocine_writer            1632
allocine_distribution      1776
jpbox_country                 0
jpbox_budget               1263
jpbox_category                0
jpbox_released_date           0
allocine_classification       0
jpbox_duration                0
jpbox_weekly_entrances        0
duration_minutes              0
dtype: int64

In [16]:
# df[df["jpbox_category"].isin(['France', 'Algérie', 'Etats-Unis'])]

df.loc[df["jpbox_category"].isin(['France', 'Algérie']), "jpbox_category"] = "Drame"
df.loc[df["jpbox_category"] == "Etats-Unis", "jpbox_category"] = "Action"

df.loc[df["allocine_fr_title"] == "Bab el-Oued City", "jpbox_duration"] = "1h33min"
df.loc[df["allocine_fr_title"] == "Blown Away", "jpbox_duration"] = "2h01min"


In [17]:
df["jpbox_category"].unique()
# df[df["jpbox_category"] == "Aventure - Action"].shape

array(['Comédie', 'Science Fiction', 'Comicbook', 'Aventure - Action',
       'Fantasy', 'Guerre', 'Animation', 'Drame', 'Western',
       'Comédie dramatique', 'Thriller', 'Péplum', 'Musical',
       'Film familial', 'Romance', 'Horreur', 'Documentaire',
       'Catastrophe', 'X - Erotique', 'Adaptation Livre', 'Court-métrage',
       'Parodie', 'Action'], dtype=object)

In [18]:
df.head(20)
# df.shape

,jpbox_fr_title,allocine_fr_title,jpbox_released_year,jpbox_actors,jpbox_directors,allocine_writer,allocine_distribution,jpbox_country,jpbox_budget,jpbox_category,jpbox_released_date,allocine_classification,jpbox_duration,jpbox_weekly_entrances,duration_minutes
0,La Ch’tite famille,La Ch’tite famille,2018,"[Dany Boon, François Berléand, Valérie Bonneto...",Dany Boon,Dany Boon,Pathé Films,France,27800000,Comédie,28/02/2018,Tout public,1h 47min,2429906,107.0
1,Rien à déclarer,Rien à déclarer,2011,"[Dany Boon, Benoît Poelvoorde, François Damien...",Dany Boon,Dany Boon,Pathé Films,France,24434009,Comédie,02/02/2011,Interdit - 10 ans,1h 48min,2587056,108.0
2,Avatar,Avatar,2009,"[Sam Worthington, Sigourney Weaver]",James Cameron,James Cameron,The Walt Disney Company France,Etats-Unis,315000000,Science Fiction,16/12/2009,Tout public,2h 40min,2648596,160.0
3,Avengers: Infinity War,Avengers: Infinity War,2018,"[Chris Evans, Chris Hemsworth, Josh Brolin, Ro...",- Russo (brothers),Christopher Markus,The Walt Disney Company France,Etats-Unis,295000000,Comicbook,25/04/2018,Tout public,2h 36min,2565953,156.0
4,"Pirates des Caraïbes, le secret du coffre maudit",Pirates des Caraïbes : le Secret du Coffre Maudit,2006,"[Johnny Depp, Orlando Bloom, Geoffrey Rush, Ke...",Gore Verbinski,Ted Elliott,Buena Vista International,Etats-Unis,225000000,Aventure - Action,02/08/2006,Interdit - 10 ans,2h 30min,2708112,150.0
5,Harry Potter et les reliques de la mort - part...,Harry Potter et les reliques de la mort - part...,2010,"[Daniel Radcliffe, Emma Watson, Helena Bonham ...",David Yates,Joanne Kathleen Rowling,None,Etats-Unis,125000000,Fantasy,24/11/2010,Tout public,2h 30min,2537450,150.0
6,Le Roi Lion (2019),Le Roi Lion,2019,"[Jamel Debbouze, Jean Reno, Seth Rogen]",Jon Favreau,Jeff Nathanson,The Walt Disney Company France,Etats-Unis,260000000,Aventure - Action,17/07/2019,Interdit - 6 ans,1h 58min,3252896,118.0
7,Harry Potter et le prisonnier d'Azkaban,Harry Potter et le Prisonnier d'Azkaban,2004,"[Daniel Radcliffe, Emma Watson, Gary Oldman, A...",Alfonso Cuaron,Steve Kloves,Warner Bros. France,Etats-Unis,130000000,Fantasy,02/06/2004,Interdit - 10 ans,2h 21min,2503699,141.0
8,Star Wars: Les derniers Jedi,Star Wars - Les Derniers Jedi,2017,"[Adam Driver, Benicio Del Toro, Joseph Gordon-...",Rian Johnson,Rian Johnson,The Walt Disney Company France,Etats-Unis,200000000,Fantasy,13/12/2017,Interdit - 8 ans,2h 31min,2510462,151.0
9,Harry Potter et l'Ordre du Phénix,Harry Potter et l'Ordre du Phénix,2007,"[Robert Pattinson, Daniel Radcliffe, Emma Wats...",David Yates,Michael Goldenberg,Warner Bros. France,Etats-Unis,150000000,Fantasy,11/07/2007,Interdit - 10 ans,2h 18min,2486291,138.0


In [ ]:
import json


df.to_csv("data/dataset_nottoyé.csv", index = False)

records = df.replace({np.nan: None}).to_dict(orient="records")
with open("data/dataset_nottoyé.json", "w", encoding="utf-8") as f:
    f.write("[\n")
    for i, record in enumerate(records):
        json_str = json.dumps(record, ensure_ascii=False)
        if i < len(records) - 1:
            f.write(f"    {json_str},\n")
        else:
            f.write(f"    {json_str}\n")
    f.write("]\n")


In [20]:
# budget_imdb = response.css("section[data-testid='BoxOffice'] div.sc-f65f65be-0.dQVJPm ul li div ul li span::text").get()

# page_recherche = response.urljoin(response.css("div.sc-b03627f1-2.gWHDBT ul li div.ipc-metadata-list-summary-item__c a.ipc-metadata-list-summary-item__t::attr('href')").get())

In [21]:
# garder tous les acteurs, créer une nouyvelle variable pour dire le nombre d'acteurs dans notre TOP 50 et TOP 100 des acteurs (2 colonnes)
# 
